In [1]:
import torch
import numpy as np

In [2]:
BOARD_SIZE = 8

Environment closely follows OpenAI gym API. Currently can not be invoked with ```gym.make("env_id")```, though it should be easy to do.

In [3]:
import gym
import gym_go

env = gym.make('gym_go:go-v0', size=BOARD_SIZE, komi=0, reward_method='real')

In [4]:
import monte_carlo_tree


class GoNode(monte_carlo_tree.Node):
    """Go Game Tree Node"""

    def prepare_state(self, player=None):
        """
        Prepare game state X from perspective of current player
        [
            [ 1 -1 -1 ]
            [ 1  0  0 ]
            [ 0  0 -1 ]
        ]

        
        Where  
            1:  current player
            -1: opposing player
            
        """

        if player == None:
            player = self.current_player()

        # take advantage of game symmetry        
        state = self.blacks() - self.whites() if player == 1 else self.whites() - self.blacks()

        return state

    def whites(self):
        """White pieces on board"""
        return self.state[0]
    
    def blacks(self):
        """Black pieces on board"""
        return self.state[1]

    def current_player(self):
        return 1 if self.state[2, 0, 0] == 1 else -1
    
    def possible_actions(self, player=None, raw=False):
        """List of possible next actions"""
        actions = (self.blacks() + self.whites()) + self.state[3]
        return np.argwhere(actions==0).tolist()

    def prepare_action(self, action):
        return action

    def evaluate(self, env):
        return env.winning()

## Random Play Tree

This is very basic algorithm that plays the game by making random moves. Sometimes it reaches the end goal, but overall it supper inneficient.

In [5]:
rt = monte_carlo_tree.RandomPlayTree(env, GoNode, BOARD_SIZE)


def random_play():
    '''
    Play a game using random tree strategy
    '''
    return rt.simulate(rt.root_node)
    

def build_stats(playfunc, n_games=100):
    '''
    Play a number of random games and display result
    '''

    black_wins = 0
    white_wins = 0
    draws = 0
    moves = []
    
    for _ in range(n_games):
        m = playfunc()

        reward = env.winning()

        if reward > 0:
            black_wins += 1
        elif reward < 0:
            white_wins += 1
        elif reward == 0.0:
            draws += 1
       
        moves.append(m.depth())
    
    print("Blacks: ", black_wins, "Whites: ", white_wins, "Draws: ", draws, "Moves mean:", np.mean(moves))

In [6]:
build_stats(random_play, 100)

Blacks:  53 Whites:  46 Draws:  1 Moves mean: 106.81


## Monte Carlo Search Tree

In [7]:
mcst = monte_carlo_tree.MonteCarloPlayTree(env, GoNode, BOARD_SIZE)

def mtsc_play():
    '''
    Play a game using MonteCarloSearchTree
    '''
    
    return mcst.simulate(mcst.root_node)

In [8]:
build_stats(mtsc_play, 100)

Blacks:  56 Whites:  43 Draws:  1 Moves mean: 104.31


## Alpha Zero

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

class ActorCritic(nn.Module):

    def __init__(self, board_size=BOARD_SIZE):
        super(ActorCritic, self).__init__()
        
        self.board_size = board_size
        self.conv1 = nn.Conv2d(1, 64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.layer1 = nn.Linear(128, 256)
        self.layer2 = nn.Linear(128, 1)
        
    def forward(self, x):

        x = x.view(-1, self.board_size, self.board_size)
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv3(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv4(x))
        x = F.max_pool2d(x, 4)[:, 0, 0]
        x = F.dropout(x, p=0.2, training=self.training)
        a = self.layer1(x)
        v = self.tanh(self.layer2(F.tanh(x)))
        
        y = x.view(-1, self.board_size**2)
        y = F.relu(self.fc(y))
        
        # Actor head
        a = F.relu(self.fc_action1(y))
        a = self.fc_action2(a)
        
        # availability of moves
        avail = (torch.abs(x)!=1).float()
        avail = avail.view(-1, self.board_size**2) 
        
        # locations where actions are not possible, we set the prob to zero
        maxa = torch.max(a)
        # subtract off max for numerical stability (avoids blowing up at infinity)
        exp = avail*torch.exp(a-maxa)
        prob = (exp/torch.sum(exp))
        
        prob = prob.view(-1, BOARD_SIZE,BOARD_SIZE)
        
        # Critic head
        value = torch.relu(self.fc_value1(y))
        value = torch.tanh(self.fc_value2(value))

        return prob, value

actor_critic_network = ActorCritic().to(device)